In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Uterine_Carcinosarcoma"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Uterine_Carcinosarcoma/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Uterine_Carcinosarcoma/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Uterine_Carcinosarcoma/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Uterine_Carcinosarcoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# Step 1: Review subdirectories to find one related to Uterine Carcinosarcoma
import os

# List all directories in TCGA root directory
tcga_dirs = os.listdir(tcga_root_dir)
print(f"Available TCGA directories: {tcga_dirs}")

# Look for directories related to Uterine Carcinosarcoma
relevant_dirs = []
for dir_name in tcga_dirs:
    dir_lower = dir_name.lower()
    if "uterine" in dir_lower and "carcinosarcoma" in dir_lower:
        relevant_dirs.append(dir_name)

print(f"Potential relevant directories for {trait}: {relevant_dirs}")

# Select the appropriate directory for the trait
selected_dir = None
if relevant_dirs:
    selected_dir = relevant_dirs[0]  # Take the first match since it's specific

if selected_dir:
    print(f"Selected directory for {trait}: {selected_dir}")
    
    # Get the full path to the directory
    cohort_dir = os.path.join(tcga_root_dir, selected_dir)
    
    # Step 2: Find clinical and genetic data files
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    print(f"Clinical data file: {clinical_file_path}")
    print(f"Genetic data file: {genetic_file_path}")
    
    # Step 3: Load the data files
    clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')
    genetic_df = pd.read_csv(genetic_file_path, index_col=0, sep='\t')
    
    # Step 4: Print column names of clinical data
    print("\nClinical data columns:")
    print(clinical_df.columns.tolist())
    
    # Check if both datasets are available
    is_gene_available = not genetic_df.empty
    is_trait_available = not clinical_df.empty
    
    # Initial validation
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )
else:
    print(f"No directory specifically relevant to the trait: {trait}")
    
    # Since the trait is not directly represented, we should record this fact
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=False,
        is_trait_available=False
    )
    print(f"Task marked as completed. {trait} is not directly represented in the TCGA dataset.")


Available TCGA directories: ['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)', 'TCGA_Colon_Cancer_(COAD)', 'TCGA_Endometrioid_Cancer_


Clinical data columns:
['CDE_ID_3226963', '_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'age_at_initial_pathologic_diagnosis', 'aln_pos_ihc', 'aln_pos_light_micro', 'anatomic_neoplasm_subdivision', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'birth_control_pill_history_usage_category', 'clinical_stage', 'colorectal_cancer', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_additional_surgery_procedure', 'days_to_new_tumor_event_after_initial_treatment', 'diabetes', 'form_completion_date', 'gender', 'height', 'histological_type', 'history_of_neoadjuvant_treatment', 'horm_ther', 'hypertension', 'icd_10', 'icd_o_3_histology', 'icd_o_3_site', 'informed_consent_verified', 'initial_pathologic_diagnosis_method', 'initial_weight', 'is_ffpe', 'lost_follow_up', 'menopause_status

### Step 2: Find Candidate Demographic Features

In [3]:
# Step 1: Identify candidate demographic features
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth']
candidate_gender_cols = ['gender']

# Step 2: Extract and preview candidate columns
clinical_data_file, _ = tcga_get_relevant_filepaths(os.path.join(tcga_root_dir, f"TCGA_{trait.replace('_', '_')}_(UCS)"))
clinical_df = pd.read_csv(clinical_data_file, sep='\t', index_col=0)

# Extract and preview age columns
if candidate_age_cols:
    age_df = clinical_df[candidate_age_cols]
    print("Age columns preview:")
    print(preview_df(age_df))

# Extract and preview gender columns
if candidate_gender_cols:
    gender_df = clinical_df[candidate_gender_cols]
    print("Gender columns preview:")
    print(preview_df(gender_df))


Age columns preview:
{'age_at_initial_pathologic_diagnosis': [65, 63, 69, 68, 61], 'days_to_birth': [-24102, -23359, -25413, -25041, -22341]}
Gender columns preview:
{'gender': ['FEMALE', 'FEMALE', 'FEMALE', 'FEMALE', 'FEMALE']}


### Step 3: Select Demographic Features

In [4]:
# Examine the age columns
age_columns = {'age_at_initial_pathologic_diagnosis': [65, 63, 69, 68, 61], 
               'days_to_birth': [-24102, -23359, -25413, -25041, -22341]}

# Examine the gender columns
gender_columns = {'gender': ['FEMALE', 'FEMALE', 'FEMALE', 'FEMALE', 'FEMALE']}

# Select appropriate columns for age and gender
# For age, we have two options:
# 1. 'age_at_initial_pathologic_diagnosis' - direct age values
# 2. 'days_to_birth' - negative days since birth (can be converted to age)
# We'll choose 'age_at_initial_pathologic_diagnosis' as it's already in years and easier to interpret
age_col = 'age_at_initial_pathologic_diagnosis'

# For gender, there's only one option available
gender_col = 'gender' if gender_columns else None

# Print the selected columns
print(f"Selected age column: {age_col}")
print(f"Age column preview: {age_columns[age_col]}")
print(f"Selected gender column: {gender_col}")
print(f"Gender column preview: {gender_columns[gender_col] if gender_col else None}")


Selected age column: age_at_initial_pathologic_diagnosis
Age column preview: [65, 63, 69, 68, 61]
Selected gender column: gender
Gender column preview: ['FEMALE', 'FEMALE', 'FEMALE', 'FEMALE', 'FEMALE']


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize clinical features
# Use the Uterine Carcinosarcoma directory identified in Step 1
selected_dir = "TCGA_Uterine_Carcinosarcoma_(UCS)"
cohort_dir = os.path.join(tcga_root_dir, selected_dir)

# Get the file paths for clinical and genetic data
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

# Load the data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)

# Extract standardized clinical features using the provided trait variable
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait=trait,  # Using the provided trait variable
    age_col=age_col, 
    gender_col=gender_col
)

# Save the clinical data to out_clinical_data_file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Saved clinical data with {len(clinical_features)} samples")

# Step 2: Normalize gene symbols in gene expression data
# Transpose to get genes as rows
gene_df = genetic_df

# Normalize gene symbols using NCBI Gene database synonyms
normalized_gene_df = normalize_gene_symbols_in_index(gene_df)
print(f"After normalization: {len(normalized_gene_df)} genes remaining")

# Save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Saved normalized gene expression data")

# Step 3: Link clinical and genetic data
# Merge clinical features with genetic expression data
linked_data = clinical_features.join(normalized_gene_df.T, how='inner')
print(f"Linked data shape: {linked_data.shape} (samples x features)")

# Step 4: Handle missing values
cleaned_data = handle_missing_values(linked_data, trait_col=trait)
print(f"After handling missing values, data shape: {cleaned_data.shape}")

# Step 5: Determine if trait or demographics are severely biased
trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait=trait)

# Step 6: Validate data quality and save cohort information
note = "The dataset contains gene expression data along with clinical information for uterine carcinosarcoma patients from TCGA."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=trait_biased,
    df=cleaned_data,
    note=note
)

# Step 7: Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Saved usable linked data to {out_data_file}")
else:
    print("Dataset was determined to be unusable and was not saved.")

Saved clinical data with 57 samples


After normalization: 19848 genes remaining


Saved normalized gene expression data
Linked data shape: (57, 19851) (samples x features)


After handling missing values, data shape: (57, 19851)
Quartiles for 'Uterine_Carcinosarcoma':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1
Max: 1
The distribution of the feature 'Uterine_Carcinosarcoma' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 62.0
  50% (Median): 68.0
  75%: 76.0
Min: 51
Max: 90
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0' with 57 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



Dataset was determined to be unusable and was not saved.
